In [1]:
import sys
print(sys.executable)

C:\Program Files\GeoScene\Pro\bin\Python\envs\arcgispro-py3\python.exe


In [5]:
!pip install PyMuPDF

  Using cached PyMuPDF-1.24.13-cp39-abi3-win_amd64.whl.metadata (3.4 kB)
Using cached PyMuPDF-1.24.13-cp39-abi3-win_amd64.whl (16.2 MB)


In [7]:
!pip install PyPDF2

In [8]:
import PyPDF2
import fitz

pdf_file_path = "E:\\下载\\EigenArchitecture.pdf"
pdf_document = fitz.open(pdf_file_path)
page = pdf_document.load_page(9)
textInfo = page.get_text('blocks',flags=1+2+8)
text = [t[4] for t in textInfo]

print(text)

['010\nCULTIVATING THE GENERIC\n', 'The Beauty of Information Technology\nComputers seem to be as universal as architecture, at least as long as they are thought \nof as abstract machines. But if, due to an improper notion of abstraction, they are per-\nceived as mere\u200a—\u200aalbeit fast\u200a—\u200amachines, they are frightening, having by now become \nsuperfast: just listen, e.g., to Paul Virilio in his War and Cinema (1989), Speed and Poli-\ntics (1986), or The Information Bomb (2000), and you cannot help get scared. Or to Jean \nBaudrillard in Carnival and Cannibal, or the Play of Global Antagonisms (2010), or asking \nWhy Hasn’t Everything Already Disappeared? (2009). Why not, indeed? Trying to slow them \ndown? Not a chance. Is that a satisfying scenario, one we’d want to play in? Or are we, \nconversely, not so much scared as fascinated by the power of computers as machines \n(i.e. not as abstract machines) and desirous to use that power for our projects? Then \nwe are in fo

In [9]:
import re

def extract_text_from_pdf(pdf_path):
    text = {}
    pdf_document = fitz.open(pdf_path)
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        textInfo = page.get_text("blocks", flags=1+2+8)
        paragraphs = [re.sub('\n', ' ', t[4]) for t in textInfo]
        text[page_num] = paragraphs
    pdf_document.close()
    return text


In [10]:
def read_pdf_paragraphs(pdf_content, ID):
    paragraphs = []
    
    for page_num, page_paragraphs in pdf_content.items():
        for paragraph in page_paragraphs:
            
            content = re.sub('\s+', ' ', paragraph)
            content = re.sub('\n', ' ', content)
            
            paragraphs.append(content.strip())
    
    paragraphs = merge_strings_until_limit(paragraphs, 200, 1000)
    
    paragraphs = [{'paragraph': paragraph, 'nr': i, 'bookID': ID} for i, paragraph in enumerate(paragraphs)]
    
    return paragraphs

def merge_strings_until_limit(strings, min_length, max_length, test_for_max = 0):
    merged_string = ""
    merged_strings = []
    
    for s in strings:
        if len(merged_string) <= min_length:
            merged_string += s
        
        elif len(merged_string) > max_length and test_for_max<5:
                splitParagraph = merged_string.split('.')
                splitParagraphRePoint = []
                for sp in splitParagraph:
                    splitParagraphRePoint.append(sp+'.')
                
                merged = merge_strings_until_limit(splitParagraphRePoint, min_length, max_length, test_for_max+1)
                merged_strings.extend(merged)
                merged_string = s
        else:
            merged_strings.append(merged_string)
            merged_string = s
    
    if merged_string:
        merged_strings.append(merged_string)
    
    return merged_strings

In [11]:
pdf_file_path = "E:\\下载\\EigenArchitecture.pdf"
output_text = extract_text_from_pdf(pdf_file_path)
prosecceed_text = read_pdf_paragraphs(output_text, '1')


In [12]:
prosecceed_text

[{'paragraph': 'Eigen ArchitectureComputability as LiteracyApplied virtuality book serieseditors Ludger Hovestadt And Vera Bühlmann042 Ekaterina AgeevaHYbridity as an Urban Speculation072 melina mezariGeneric villa088 diana alvarez',
  'nr': 0,
  'bookID': '1'},
 {'paragraph': 'Exiled to the Virtual World or the Inconsistency of the Real058 Bojana MiskeljinEigenwindow As A Reflection Of Singaporeans’ Cultural Diversity010 Miro RomanFour Chairs and all the others028 Stanislava Pedrojevic',
  'nr': 1,
  'bookID': '1'},
 {'paragraph': 'Hard-Boiled World Wide Web And The End Of Distance108 mauricio rodriguezA House of Things007 Ludger HovestadtCultivating the generic069 Vera BühlmannArticulating a thing entirely in its own termsEigen Architecture',
  'nr': 2,
  'bookID': '1'},
 {'paragraph': 'Computability as LiteracyApplied virtuality book serieseditors Ludger Hovestadt And Vera Bühlmann007Ludger Hovestadt Cultivating the genericA mathematically inspired pathway for architectsCULTIVATING 

In [13]:
!pip install pymupdf scikit-learn

In [33]:
import fitz  # PyMuPDF
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import nltk

nltk.download('wordnet')
nltk.download('stopwords')

STOP_WORDS = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
def simple_preprocess(text, min_len=3, deacc=True):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    words = text.split()
    if deacc:
        words = [re.sub(r'[^a-zA-Z0-9\s]', '', word) for word in words]
    words = [word for word in words if len(word) >= min_len]
    return words

def process_paragraphs(paragraphs):
    processed_paragraphs = []
    for paragraph in paragraphs:
        words = simple_preprocess(paragraph, min_len=3, deacc=True)
        lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
        filtered_words = [word for word in lemmatized_words if word.lower() not in STOP_WORDS and word.isalpha()]
        stemmed_words = [stemmer.stem(word) for word in filtered_words]
        processed_paragraph = " ".join(stemmed_words)
        processed_paragraphs.append(processed_paragraph)
    return processed_paragraphs

def extract_text_and_images(pdf_path):
    doc = fitz.open(pdf_path)
    text_list = []
    images = []
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        text_list.append(text)
        image_list = page.get_images(full=True)
        for image_index, img in enumerate(image_list):
            xref = img[0]
            base_image = doc.extract_image(xref)
            if base_image:
                image_bytes = base_image["image"]
                images.append(image_bytes)
    return text_list, images

def tfidf_vectorization(text_list):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(text_list)
    
    svd = TruncatedSVD(n_components=100)  
    reduced_tfidf_matrix = svd.fit_transform(tfidf_matrix)
    
    return vectorizer, reduced_tfidf_matrix, svd  

def search_images(processed_query, vectorizer, reduced_tfidf_matrix, svd):
    query_vector = vectorizer.transform([processed_query])  
    reduced_query_vector = svd.transform(query_vector)  
    cosine_similarities = cosine_similarity(reduced_query_vector, reduced_tfidf_matrix).flatten()
    most_relevant_page_index = cosine_similarities.argmax()
    return most_relevant_page_index


In [52]:
pdf_path = "E:\\下载\\EigenArchitecture.pdf"
text_list, images = extract_text_and_images(pdf_path)


processed_text_list = process_paragraphs(text_list)


vectorizer, reduced_tfidf_matrix, svd = tfidf_vectorization(processed_text_list)


query = "Churches and fractals"
processed_query = simple_preprocess(query, min_len=3, deacc=True)
lemmatized_query = [lemmatizer.lemmatize(word) for word in processed_query]
filtered_query = [word for word in lemmatized_query if word.lower() not in STOP_WORDS and word.isalpha()]
stemmed_query = [stemmer.stem(word) for word in filtered_query]
processed_query = " ".join(stemmed_query)  


most_relevant_page_index = search_images(processed_query, vectorizer, reduced_tfidf_matrix, svd)

print(f"The most relevant image is on page {most_relevant_page_index + 1}.")

The most relevant image is on page 63.


In [51]:
n_samples, n_features = tfidf_matrix.shape
print(f"The TF-IDF matrix has shape: {n_samples} samples and {n_features} features.")

The TF-IDF matrix has shape: 128 samples and 100 features.
